# Transcript Data Processing
This notebook processes presidential address transcript files. This includes filtering, cleaning and exporting the dataframe for topic modeling.

In [1]:
import pandas as pd
import re

# Process Data

In [1]:
import sys

In [2]:
sys.version

'3.6.5 |Anaconda custom (64-bit)| (default, Apr 29 2018, 16:14:56) \n[GCC 7.2.0]'

## Load Data

In [2]:
trans_df = pd.read_csv('data/transcripts.csv', index_col=0)
trans_df.head()

,Title,Date,President,Transcript
0,"February 5, 2019: State of the Union Address","February 05, 2019",Donald Trump,"Transcript\nMadam Speaker, Mr. Vice President,..."
1,"January 19, 2019: Remarks about the US Souther...","January 19, 2019",Donald Trump,Transcript\nTHE PRESIDENT: Just a short time a...
2,"September 25, 2018: Address at the 73rd Sessio...","September 25, 2018",Donald Trump,"Transcript\nTHE PRESIDENT: Madam President, Mr..."
3,"July 24, 2018: Speech at the Veterans of Forei...","July 24, 2018",Donald Trump,"Transcript\nTHE PRESIDENT: Thank you, Lee. Tha..."
4,"March 19, 2018: Remarks on Combating the Opioi...","March 19, 2018",Donald Trump,Transcript\nTHE PRESIDENT: Thank you to our Fi...


In [3]:
# additional transcripts
trans_add = pd.read_csv('data/transcripts_add.csv', index_col=0)

In [4]:
# merge transcripts
speech_df = pd.concat([trans_df, trans_add], axis=0, ignore_index=True).copy()

In [5]:
# number of scraped speech transcripts
len(speech_df)

1012

## Check Data
Check for missing data and duplicates

In [8]:
# check for missing data in columns
print('empty dates:',speech_df['Date'].isnull().sum())
print('empty presidents:', speech_df['President'].isnull().sum())
print('empty transcripts:', speech_df['Transcript'].isnull().sum())

empty dates: 0
empty presidents: 0
empty transcripts: 0


In [9]:
# check for duplicates
speech_df[speech_df.duplicated(keep=False)]

,Title,Date,President,Transcript
623,"June 12, 1895: Declaration of US Neutrality","June 12, 1895",Grover Cleveland,Transcript\nBy the President of the United Sta...
624,"June 12, 1895: Declaration of US Neutrality","June 12, 1895",Grover Cleveland,Transcript\nBy the President of the United Sta...


In [11]:
# remove duplicates
speech_df.drop_duplicates(inplace=True)

In [12]:
# check speakers
speech_df['President'].unique()

array(['Donald Trump', 'Barack Obama', 'George W. Bush', 'Bill Clinton',
       'George H. W. Bush', 'Ronald Reagan', 'Chester A. Arthur',
       'Jimmy Carter', 'Gerald Ford', 'Richard M. Nixon',
       'Lyndon B. Johnson', 'John F. Kennedy', 'Dwight D. Eisenhower',
       'Harry S. Truman', 'Franklin D. Roosevelt', 'Herbert Hoover',
       'Calvin Coolidge', 'Warren G. Harding', 'Woodrow Wilson',
       'William Taft', 'Theodore Roosevelt', 'William McKinley',
       'Grover Cleveland', 'Benjamin Harrison', 'James A. Garfield',
       'Rutherford B. Hayes', 'Ulysses S. Grant', 'Andrew Johnson',
       'Abraham Lincoln', 'James Buchanan', 'Franklin Pierce',
       'Millard Fillmore', 'Zachary Taylor', 'James K. Polk',
       'John Tyler', 'William Harrison', 'John Quincy Adams',
       'Martin Van Buren', 'Andrew Jackson', 'James Monroe',
       'James Madison', 'Thomas Jefferson', 'John Adams',
       'George Washington'], dtype=object)

## Filter/Format Data
Remove undesired data types and sort data by date

In [13]:
# filter out debate transcripts
speech_df = speech_df[~speech_df['Title'].str.contains('Debate')]

In [14]:
# reformat dates to datetime
speech_df['Date'] = pd.to_datetime(speech_df['Date'])

In [15]:
# sort by date
speech_df.sort_values(by=['Date'], inplace=True)
# reset index based on earliest date
speech_df.reset_index(drop=True, inplace=True)

In [16]:
speech_df.head()

,Title,Date,President,Transcript
0,"April 30, 1789: First Inaugural Address",1789-04-30,George Washington,Transcript\nFellow Citizens of the Senate and ...
1,"October 3, 1789: Thanksgiving Proclamation",1789-10-03,George Washington,Transcript\nWhereas it is the duty of all Nati...
2,"January 8, 1790: First Annual Message to Congress",1790-01-08,George Washington,Transcript\nFellow Citizens of the Senate and ...
3,"December 8, 1790: Second Annual Message to Con...",1790-12-08,George Washington,Transcript\nFellow citizens of the Senate and ...
4,"December 29, 1790: Talk to the Chiefs and Coun...",1790-12-29,George Washington,Transcript\nI the President of the United Stat...


## Clean Text
Remove unnecessary words throughout text data

In [17]:
# function to clean transcript text
def text_cleaner(drty_txt):
    # remove newline characters ("\n")
    cln_txt = drty_txt.replace('\n', ' ')
    # remove leading "Transcript" text
    if cln_txt.startswith('Transcript'):
        cln_txt = cln_txt[len('Transcript '):]
    # remove notes in parantheses throughout transcript
    cln_txt = re.sub(r'\([^)]*\)', '', cln_txt)
    # return cleansed text
    return(cln_txt)

In [18]:
# run transcripts through cleaner function
speech_df['Transcript'] = speech_df['Transcript'].apply(lambda x: text_cleaner(x))

In [19]:
speech_df.head()

,Title,Date,President,Transcript
0,"April 30, 1789: First Inaugural Address",1789-04-30,George Washington,Fellow Citizens of the Senate and the House of...
1,"October 3, 1789: Thanksgiving Proclamation",1789-10-03,George Washington,Whereas it is the duty of all Nations to ackno...
2,"January 8, 1790: First Annual Message to Congress",1790-01-08,George Washington,Fellow Citizens of the Senate and House of Rep...
3,"December 8, 1790: Second Annual Message to Con...",1790-12-08,George Washington,Fellow citizens of the Senate and House of Rep...
4,"December 29, 1790: Talk to the Chiefs and Coun...",1790-12-29,George Washington,"I the President of the United States, by my ow..."


# Address Type
Label addresses by type

In [32]:
# create column for address type
speech_df.insert(3, 'Type', 'other')

In [34]:
# fireside chats
speech_df.loc[speech_df['Title'].str.contains('fireside chat', case=False), 'Type'] \
= 'fireside'
# address to nation
speech_df.loc[speech_df['Title'].str.contains('to the nation ', case=False), 'Type'] \
= 'national'
# convention
speech_df.loc[speech_df['Title'].str.contains('convention', case=False), 'Type'] \
= 'convention'
# inaugural
speech_df.loc[speech_df['Title'].str.contains('inaugural', case=False), 'Type'] \
= 'inauguration'
# state of the union
speech_df.loc[speech_df['Title'].str.contains('annual message|state of the union', case=False), 'Type'] \
= 'state_union'

In [25]:
speech_df['Title'].str.contains('inaugural', case=False).sum()

57

In [28]:
speech_df['Title'].str.contains('annual message|state of the union', case=False).sum()

201

In [20]:
speech_df['Title'].str.contains('national convention|nomination', case=False).sum()

36

In [21]:
speech_df['Title'].str.contains('farewell', case=False).sum()

15

In [22]:
speech_df['Title'].str.contains('fireside chat', case=False).sum()

30

In [23]:
speech_df['Title'].str.contains('to the nation ', case=False).sum()

32

# Export Clean Data

In [38]:
speech_df.head()

,Title,Date,President,Type,Transcript
0,"April 30, 1789: First Inaugural Address",1789-04-30,George Washington,inauguration,Fellow Citizens of the Senate and the House of...
1,"October 3, 1789: Thanksgiving Proclamation",1789-10-03,George Washington,other,Whereas it is the duty of all Nations to ackno...
2,"January 8, 1790: First Annual Message to Congress",1790-01-08,George Washington,state_union,Fellow Citizens of the Senate and House of Rep...
3,"December 8, 1790: Second Annual Message to Con...",1790-12-08,George Washington,state_union,Fellow citizens of the Senate and House of Rep...
4,"December 29, 1790: Talk to the Chiefs and Coun...",1790-12-29,George Washington,other,"I the President of the United States, by my ow..."


In [36]:
# total number of addresses
len(speech_df)

998

In [37]:
speech_df.to_csv('data/cln_transcripts.csv')